# In this notebook

- we try to find the close attributes for a given set of attributes 
- we try to cluster attributes 
    - NOTE: we can't use unsupervised clustering because we don't know how many clusters can be there 
    - we first find a 5 or 10 nearest neighbors for each attribute
        - we then group/merge attributes together into clusters based on (and/or)
            - mean distance between neighbors across all neighbor sets
            - occurrence frequency of nearest neighbors


In [1]:
import pandas as pd
import json

pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 200
pd.options.display.max_rows = 1000

In [2]:
from sqlalchemy import create_engine
import psycopg2 
import io

In [6]:
import os
import glob

In [11]:
import pickle

In [17]:
import numpy as np

# Load intermediate save files

In [75]:
save_files = sorted(glob.glob("*pkl"))

In [76]:
save_files

['cumulative_subset-000.pkl',
 'cumulative_subset-001.pkl',
 'cumulative_subset-002.pkl',
 'cumulative_subset-003.pkl',
 'cumulative_subset-004.pkl',
 'cumulative_subset-005.pkl',
 'cumulative_subset-006.pkl',
 'cumulative_subset-007.pkl']

In [77]:
# intermediate save files
key_phrases = pd.read_pickle(save_files[-1])
    

In [78]:
key_phrases.shape

(6686, 5)

## Set up the data frame

In [79]:
key_phrases['reviews'].apply(lambda r: len(r)).sum()

18975

In [80]:
key_phrases['category'] = 'Monitor'
key_phrases['n_reviewers'] = key_phrases['reviewers'].apply(lambda r: len(r))
key_phrases['n_reviews'] = key_phrases['reviews'].apply(lambda r: len(r))
key_phrases['reviewer_idf'] = np.log(45480/key_phrases['n_reviewers'])

print("Warning: magic number in this code block")

In [81]:
key_phrases['key_phrase_id'] = list(range(key_phrases.shape[0]))
key_phrases = key_phrases.reset_index().rename(columns={'index': 'phrase'})[[ 
    'key_phrase_id', 'phrase', 'reviews', 'reviewers', 'products', 'n_positive', 'n_negative', 'category', 'reviewer_idf'
]]

In [82]:
key_phrases.head()

,key_phrase_id,phrase,reviews,reviewers,products,n_positive,n_negative,category,reviewer_idf
0,0,game room,{159566},{A3EY8AM4N0KJ5B},{B001DKMO0A},1,0,Monitor,10.725028
1,1,easy,"{225794, 247314, 333848, 312858, 232475, 177691, 247324, 225821, 246826, 246829, 274991, 333871, 177715, 247350, 177720, 265273, 265276, 333884, 177727, 234050, 333890, 246852, 265288, 247369, 275...","{A2PW29SY7QZ6YM, ABVRKJVHCU1S9, AKZ897256POEL, A3T2PH4TNHU91V, A2JRQJRZN0HZCI, A2SURI17Z9D3XP, APST4MEN1GLI0, A1JQISXQ7EP9RE, A3O4CWUFSGABNE, A2DIFZ1OF8P2ZF, A2OPXFCOKHR0G6, A3MEMX0XF00TBD, A3N1BU...","{B003D59FDW, B0037W687M, B004B0H5R6, B001QSTG9G, B005BZND6M, B00522X8B4, B001O9EUSO, B003Y73Q4S, B004ZMH65Q, B004LQ1QDE, B003UYUIG4, B001P5CAV6, B004Y3FE6O, B002WWRC7I, B003CYL7O4, B003Y3BJ7S, B00...",146,8,Monitor,5.688075
2,2,good product,"{315140, 315152, 314387, 178455, 232472, 314268, 246691, 315171, 333732, 168489, 246574, 177713, 265268, 322996, 284484, 159566, 225744, 188114, 234070, 232664, 314330, 333786, 232796, 275036, 180...","{A2AXVO55093S3D, AEB4X93X11US7, AWX1PY9ZZK73Z, A3MHHO0IY13KFV, ALVMXV9RFRNPT, A22QN8JDZS8YOF, A2PZY48K3G9YCU, A3MTBU6HGWKVRT, A17A0VVCBH2ZQT, AKT799RG73ZIV, AZHBTMK11G9DO, A2KEX02KHNGDUD, A2OPFHAT...","{B003D59FDW, B004B0H5R6, B001H1CD92, B003Y73Q4S, B004Y3FE6O, B003CYL7O4, B003Y3BJ7S, B0052HRSJW, B005C29QN8, B002453K5G, B001DKMO0A, B005C24OP8, B001PLEHQG, B003FP7OYM, B0036ISPSW, B001LYWBOM, B00...",31,4,Monitor,7.169680
3,3,stars,"{333827, 321542, 321545, 321555, 333845, 333846, 321559, 321561, 333871, 333877, 333879, 333880, 333886, 333892, 333906, 333909, 333910, 333913, 333915, 333922, 313518, 313522, 313523, 313526, 313...","{A20PNAORUEDXJM, A388M183NRTQI9, A38X7DM3V3W6WR, ATDFGUFRH1OZW, A30A7JFQVXJDIP, ANFGY34SKXSD2, A2W255X9KXLK9Z, A3G7K2NQS25N49, A1N70RVVQA0L1D, A208O435GWDHRM, A2DN6CTRX35B4X, A3NWBBB5LD902I, A24G2...","{B003D59FDW, B004C4WTJA, B0037W687M, B004B0H5R6, B004DELTKE, B003Y73Q4S, B002E31136, B004LQ1QDE, B004Y3FE6O, B002WWRC7I, B0039648BO, B003Y3BJ7S, B001UHOX2I, B0051156TS, B004HMIBAI, B0038IWQ7Q, B00...",411,53,Monitor,4.604731
4,4,thanks,"{281477, 247048, 334478, 314259, 334487, 314274, 246574, 234048, 323012, 171974, 159567, 225758, 246750, 185567, 246627, 225653, 233978, 306301, 314878, 234239}","{A2QO91757WRY7Q, A3TRRQ6V0UW5QA, AX4PXKR06TGLP, A2HZ2SBRT2T2IK, A3TBQB7UZ6SVBR, A1E8KCG75TEEEG, A54WHF450I5MF, A1AZ60IKL5PWPM, A1ZC69BK94HMVH, A1PR38JR6WZKI7, A2DN6CTRX35B4X, A35V86K4HDB398, AMU3L...","{B001DKMO0A, B003HFG6S0, B004HJ7JAE, B001L4AZWC, B00522X8B4, B001UHOX2S, B003FP7OYM, B0036ISPSW, B005C76ZXC, B0036C7IMM, B004Y3FE6O, B005680BX2, B003Y3BJ7S, B0051GN8JI}",18,2,Monitor,7.729296


In [83]:
key_phrases[key_phrases['phrase'] == 'image']

,key_phrase_id,phrase,reviews,reviewers,products,n_positive,n_negative,category,reviewer_idf
871,871,image,"{294017, 247044, 246917, 316935, 222087, 243977, 334474, 185595, 178192, 247057, 199831, 265368, 306329, 265372, 247068, 234271, 234019, 265380, 246693, 265381, 265383, 265256, 188080, 234032, 209...","{A3FW6SCLPCNOLM, A2IURNDS90OQ6O, A39JVI19WKKMVX, A2KX33M7CIDKOV, AQEP8LX3X7DHI, A1FN3UROHHW18T, A21GUW0ZW08WSL, A3S8MYETHEO2D0, A1ZUSBK1VCDF0F, A1UF8P8ZXXQYWQ, A43D69V657P2F, A1963SA8X835YL, A7HPQ...","{B001P5CAV6, B002DHS398, B004Y3FE6O, B004TPTX0U, B0030CE7EK, B003Y3BJ7S, B003UT2C4U, B002453K5G, B004QNHNWK, B001UHOX2S, B003FP7OYM, B0045IIZKU, B003HFG6S0, B004WK3R4U, B00535C50A, B001QVLD8A, B00...",27,9,Monitor,7.141509


# Build a nearest neighbor model for phrases

In [84]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [85]:
from sklearn.neighbors import NearestNeighbors

In [86]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [87]:
# get spacy vectors for attributes

phrase_vectors = []

key_phrases['phrase'].progress_apply(lambda p: phrase_vectors.append(nlp(p).vector))

  0%|          | 0/6686 [00:00<?, ?it/s]

0       None
1       None
2       None
3       None
4       None
        ... 
6681    None
6682    None
6683    None
6684    None
6685    None
Name: phrase, Length: 6686, dtype: object

In [88]:
#phrase_vectors[0]

In [89]:
phrase_vectors_arr = np.vstack(phrase_vectors)

In [90]:
phrase_vectors_arr.shape

(6686, 96)

In [91]:
nn = NearestNeighbors()
nn.fit(phrase_vectors_arr)

NearestNeighbors()

In [92]:
q = nlp('Easy to set up').vector

In [93]:
nn.kneighbors([q], n_neighbors=5, return_distance=False)

array([[3573,  394, 6068, 2076, 4722]])

In [94]:
def get_nearest_attributes(attribute, k=5):
    q = nlp(attribute).vector
    neighbors = nn.kneighbors([q], n_neighbors=k, return_distance=False)
    return neighbors[0]

In [97]:
qterms = ['Easy to set up', 'Quality display', 
            'Good color quality', 'Quality build', 
            'Sound quality', 
            'Minimal glare', 'Lightweight', 
            'Good viewing angles', 'Fast']

In [99]:
for q in qterms:
    print(q)
    neighbors =  get_nearest_attributes(q, 10)
    display(key_phrases.iloc[neighbors])
    

Easy to set up


,key_phrase_id,phrase,reviews,reviewers,products,n_positive,n_negative,category,reviewer_idf
3573,3573,bad tn panel,{243918},{A2I8LFSN2IS5EO},{B003UT2C4U},0,1,Monitor,10.725028
394,394,monitor just--worked,{172867},{A27VJIWHVF94DF},{B001LQXLGC},1,0,Monitor,10.725028
6068,6068,macbook friendly,{313156},{A3UHS4GW81DEDR},{B0050IE1O8},0,1,Monitor,10.725028
2076,2076,nice tn panel,{209606},{A2JRNHAVRTA52E},{B002MT6SDU},1,0,Monitor,10.725028
4722,4722,touch aware os,{276918},{AB80WF7XB7QU0},{B004C4WTJA},1,0,Monitor,10.725028
5026,5026,view screen good,{280509},{A1VX8O2E41ZXHR},{B004G8QSTO},1,0,Monitor,10.725028
5855,5855,hands free calling,{314331},{AB4LI44ZT2SX5},{B0051GN8JI},1,0,Monitor,10.725028
2745,2745,passive dp,{227229},{A1VKZNU0UQ2YAH},{B0039648BO},0,1,Monitor,10.725028
5924,5924,high res device,{313518},{A15VYABIU02FGP},{B0051156TS},1,0,Monitor,10.725028
2603,2603,pressure sensitive display--but,{226957},{AMBFK6N4JS5Z2},{B0038PQCQK},1,0,Monitor,10.725028


Quality display


,key_phrase_id,phrase,reviews,reviewers,products,n_positive,n_negative,category,reviewer_idf
3756,3756,great picture display,{246659},{A19LPHM3EM4V4B},{B003Y3BJ7S},1,0,Monitor,10.725028
5343,5343,hotel cable,{294013},{A3ARM0QXPX2LZ2},{B004TPTX0U},1,0,Monitor,10.725028
4008,4008,great value monitor,{246781},{A2H4TWGVULW3Q},{B003Y3BJ7S},1,0,Monitor,10.725028
4107,4107,screen display,"{281484, 246919}","{A1LB7ZGV5CIENB, ARG1MSOEMWDTL}","{B003Y3BJ7S, B004HJ7JAE}",0,2,Monitor,10.031881
2784,2784,pc display,{227244},{A1O2KST0G3G7FJ},{B0039648BO},1,0,Monitor,10.725028
1705,1705,great price point,{188165},{AVA8EWZIPA3N5},{B002453K5G},1,0,Monitor,10.725028
1060,1060,background display,{177691},{A2SURI17Z9D3XP},{B001PLEHRA},1,0,Monitor,10.725028
6413,6413,playstation display,{333152},{A2978FE49QQ26U},{B005C24OP8},1,0,Monitor,10.725028
5688,5688,simple grid tool,{310665},{A2OS3TIVAKUAHG},{B004ZKH1NA},1,0,Monitor,10.725028
6645,6645,good color display,{334466},{A1PB9FGX838GR},{B005C76ZXC},0,1,Monitor,10.725028


Good color quality


...

,key_phrase_id,phrase,reviews,reviewers,products,n_positive,n_negative,category,reviewer_idf
1349,1349,dad,"{305580, 180109}","{A39L2TPFYEHNXX, A1GAMGCJD407G8}","{B001RJU4TA, B004XNLD8I}",1,1,Monitor,10.031881
978,978,lo traje,{178448},{A36J3K929QDJF2},{B001QXDSS6},1,0,Monitor,10.725028
3611,3611,refurb,{243988},{A145P764RX7Q6V},{B003UT2C4U},0,1,Monitor,10.725028
4999,4999,iphoto,{280519},{A2I303VSZPSEM2},{B004G8QSTO},1,0,Monitor,10.725028
6504,6504,tension,"{333617, 333923}","{A1XLRFKBDLL672, A1DDPLXC88H3LL}",{B005C29QN8},2,0,Monitor,10.031881
1961,1961,epic fail,{202692},{AYUG1ANCJY4XH},{B002E36GWC},0,1,Monitor,10.725028
3432,3432,magenta,{247249},{A26TNQOSJQHZ6J},{B003Y3BJ7S},0,1,Monitor,10.725028
3343,3343,vx225,{247161},{AKYDGCKCY7H9F},{B003Y3BJ7S},1,0,Monitor,10.725028
3915,3915,record straight,{247462},{A8ZEIMTD8NDOV},{B003Y73Q4S},1,0,Monitor,10.725028
5039,5039,zuiko glass,{280513},{AF7OI2CM20HQ6},{B004G8QSTO},1,0,Monitor,10.725028


Good viewing angles


,key_phrase_id,phrase,reviews,reviewers,products,n_positive,n_negative,category,reviewer_idf
3088,3088,different viewing angles,{232598},{A2DS3K1Z69MGO0},{B003D1CFHY},1,0,Monitor,10.725028
2928,2928,bad viewing angles,{232745},{A1BJ78YK3AY06E},{B003D59FDW},0,1,Monitor,10.725028
4038,4038,great viewing screen,{246805},{A1HBOD86E6VSGF},{B003Y3BJ7S},1,0,Monitor,10.725028
5757,5757,huge viewing surface,{313176},{A3H7NPNDMGLOU4},{B0050IE1O8},1,0,Monitor,10.725028
6526,6526,good tilting mechanism,{333691},{A2KT270XBWEBQT},{B005C29QN8},1,0,Monitor,10.725028
663,663,great looking screen,{173397},{A2YS78CWZRFYPV},{B001LYWBOM},1,0,Monitor,10.725028
4020,4020,nice looking display,{246795},{A3V2DRC1RRM8TO},{B003Y3BJ7S},1,0,Monitor,10.725028
2663,2663,great looking unit,{226052},{A1PPMCPO6GNQHJ},{B0037WRJ4S},1,0,Monitor,10.725028
6058,6058,-great viewing angles,{313145},{A1T5EPHQUSXFB9},{B0050IE1O8},1,0,Monitor,10.725028
6318,6318,audio recording capabilities,{323005},{A3B9X4B903HJOT},{B005680BX2},0,1,Monitor,10.725028


Fast


,key_phrase_id,phrase,reviews,reviewers,products,n_positive,n_negative,category,reviewer_idf
1517,1517,dual,{185585},{A187A91P19VK7J},{B001UHOX2S},1,0,Monitor,10.725028
1069,1069,fast,"{333709, 296398, 209619, 314325, 265208, 177691, 280509}","{A1VX8O2E41ZXHR, A36OK8RFP478MK, A2BXWA9XYI7MMJ, A2SURI17Z9D3XP, A22N9HBLFBXCBH, A1AMYI63702RLA, A1EOTB1WHLSW6G}","{B004G8QSTO, B001PLEHRA, B004VFFIZC, B002MT6SDU, B005C29QN8, B0045IIZKU, B0051GN8JI}",7,0,Monitor,8.779118
5597,5597,real,"{306300, 322988}","{A27181AF6YYMRF, A1FMIQSZ9RYXW6}","{B005680BX2, B004Y3FE6O}",2,0,Monitor,10.031881
2104,2104,long,"{280457, 213667, 333563}","{A3A87S023OM0X, A2BZAXY8F5GIEU, A2L1WD9Q6AP6G0}","{B004G8QSTO, B005C29QN8, B002R0JJY4}",3,0,Monitor,9.626416
291,291,little,"{246657, 307858, 169240, 226974, 333858, 333860, 166566, 333227, 315181, 180014, 331056, 280505, 247236, 294010, 202715, 333020, 176225, 333032, 247022, 234230, 178426}","{AQ3E2VTNX5GXG, AKPTQUDBDL8OF, A2P552C32IZMTQ, A31GOE0Y2JM17C, A19JGXVVSX4MSN, A11D6TCQE1P0ZG, ABNGXUZB04PON, A2PGO1AH5Q4NT3, A1XSPKR7LLMZQ5, A1CHWLTC0TPPTM, A248KI5HF34DMS, AMIJ6W24OKETQ, A21PERY...","{B004G8QSTO, B001QX6U5E, B003HFG6S0, B005C24OP8, B001O9EUSO, B001RJQHZK, B001GFIMDU, B005B9A2DU, B002E31136, B005C29QN8, B001IBHF5S, B0052HRSJW, B0038PQCQK, B004TPTX0U, B004YCMEJU, B003Y3BJ7S}",18,3,Monitor,7.680506
6283,6283,muddy,{333078},{A3RMD31Z83T0YH},{B005C2A2W2},0,1,Monitor,10.725028
606,606,nice,"{265220, 233990, 316935, 246794, 314384, 202769, 239122, 314385, 177691, 312862, 232478, 177699, 265259, 234034, 246840, 234053, 265309, 246890, 234091, 313466, 246911, 265353, 306322, 333971, 334...","{A3212H6SF9W1LV, A1YNUQN4TSQUMX, AQC4VBZJBA6HY, A1UQK2YO0RLGK5, A26JTCPCI3P9WR, A2SURI17Z9D3XP, A1ZTQYRWXEE1P8, A3DCBB5U1JV2P5, AA86JVS21K3SJ, A3DGJMO6VPG6FJ, A3BB4EY2BTZV14, AU0UE86BCGA0P, A28JKD...","{B003D59FDW, B0037W687M, B003Y73Q4S, B002E31136, B003PBVJ7O, B004Y3FE6O, B004M905TQ, B001UHOX28, B003C2B1O2, B003CYL7O4, B003Y3BJ7S, B004G8QSTO, B001PLEHRA, B0051156TS, B0038IWQ7Q, B005B9A2DU, B00...",88,7,Monitor,6.171151
474,474,big,"{284420, 247175, 247176, 246925, 247061, 307861, 229280, 333866, 246833, 177714, 209587, 333874, 280501, 333875, 333876, 176702, 246594, 333895, 310728, 323018, 310731, 188115, 333909, 235238, 223...","{A1B6T5JZXLGSNX, AZVCNOOQK36DH, A183Y5EVMU53BI, AGGVGGB6U2OHI, A2DTB2KZZ81UEA, A2ADUB9D8PNTO3, A1CTVZ3W8XZWZ9, A3B8CB5VOGBMNV, A22AT8H08608EC, AI0BCEWRE04G0, A2T9F70ZD0ARRE, A1F0KPA232MTYC, A1WM2Q...","{B004G8QSTO, B001PLEHQG, B001LYUILK, B003B4D5TU, B005C29QN8, B002MT6SDU, B0032ANCP0, B004ZMH65Q, B001P3NU5S, B005680BX2, B004YCMEJU, B002453K5G, B004K6O3C6, B003Y3BJ7S, B003JLVLIW}",21,5,Monitor,7.466931
5227,5227,esp,{284485},{A1B8FDERWV5W0J},{B004K6O3C6},1,0,Monitor,10.725028
3393,3393,little cheap,{247201},{AGKWA6UF45EFI},{B003Y3BJ7S},1,0,Monitor,10.725028


# Trying with an unsupervised clustering model